In [5]:
import numpy as np
import pandas as pd
from simulation_modified import Grid, Source, GaussianPuff
import numpy as np
import pandas as pd
# from yourpackage import Grid, Source, GaussianPuff   # <- keep your own import

# ------------------------------------------------------------------
# grid
x_grid = np.linspace(-100, 100, 21)
y_grid = np.linspace(-100, 100, 21)
z_grid = np.array([1.8])                # single height level
grid = Grid(x_grid, y_grid, z_grid)

# ------------------------------------------------------------------
# leak-rate schedule (kg s-1) – one value for every second 0-10 s
rate = pd.Series(
    [100.50, 10.20, 0.80, 0.50, 0.30,
      0.10, 0.05, 0.02, 0.00, 0.00, 0.00],
    index=range(11)                     # times 0-10 s
)

# ------------------------------------------------------------------
# source now gets the *Series* rate
source = Source(x=0, y=0, z=1, rate=rate)

# ------------------------------------------------------------------
# simple two-point met data (changes at t = 0 s and t = 10 s)
atm = pd.DataFrame(
    {'Wind Direction': [45, 60],
     'Wind Speed':     [1.2, 1.0],
     'Stability Class':['A',  'A']},
    index=[0, 10]
)

# ------------------------------------------------------------------
# run the Gaussian-puff model
gauss_puff = GaussianPuff(grid=grid,
                          source=source,
                          atm=atm,
                          tpuff=1,          # 1-second puffs
                          tend=10,          # simulate 0-10 s
                          tstep=1)          # report every second

signal = gauss_puff.conc   # concentrations DataFrame





In [ ]:
import numpy as np
import pandas as pd
from simulation_modified import Grid, Source, GaussianPuff   # your patched code

# ------------------------------------------------------------------
# grid
x_grid = np.linspace(-100, 100, 21)
y_grid = np.linspace(-100, 100, 21)
z_grid = np.array([1.8])
grid = Grid(x_grid, y_grid, z_grid)

# ------------------------------------------------------------------
# ***constant*** leak-rate (kg s-1)
const_rate = 1.5        # ← scalar
source_A = Source(x=0, y=0, z=1, rate=const_rate)

# ------------------------------------------------------------------
atm = pd.DataFrame(
    {'Wind Direction': [45, 60],
     'Wind Speed':     [1.2, 1.0],
     'Stability Class':['C',  'D']},
    index=[0, 10]                           # times in seconds
)

# ------------------------------------------------------------------
gpuff_A = GaussianPuff(grid=grid,
                       source=source_A,
                       atm=atm,
                       tpuff=1,
                       tend=10,
                       tstep=1)

conc_A = gpuff_A.conc                      # DataFrame


In [16]:
import numpy as np
import pandas as pd
from simulation_modified import Grid, Source, GaussianPuff   # patched code

# grid -----------------------------------------------------------------
x_grid = np.linspace(-100, 100, 21)
y_grid = np.linspace(-100, 100, 21)
z_grid = np.array([1.8])
grid_mod = Grid(x_grid, y_grid, z_grid)

# constant release-rate ------------------------------------------------
rate_const = 1e-6                                  # kg s-1
rate_vector = pd.Series(np.repeat(rate_const, 11), index=range(11))  # 0–10 s
source_mod  = Source(0, 0, 1, rate_vector)


# met data -------------------------------------------------------------
atm = pd.DataFrame({'Wind Direction': [45, 60],
                    'Wind Speed':     [1.2, 1.0],
                    'Stability Class':['C',  'D']},
                   index=[0, 10])

# run patched Gaussian-puff -------------------------------------------
gpuff_mod = GaussianPuff(grid_mod, source_mod, atm, tpuff=1, tend=10)
gpuff_mod.run(grid_mod, 1)            # 1-s reporting step
conc_mod = gpuff_mod.conc



In [17]:
import chama

# You can call the classes directly from chama.simulation (per docs)
grid_ch = chama.simulation.Grid(x_grid, y_grid, z_grid)
source_ch = chama.simulation.Source(0, 0, 1, rate_const)

gpuff_ch = chama.simulation.GaussianPuff(grid_ch, source_ch, atm,
                                         tpuff=1, tend=10)
gpuff_ch.run(grid_ch, 1)
conc_ch = gpuff_ch.conc


In [18]:
import numpy as np
import pandas as pd

# align rows on (X, Y, Z, T) so we compare the same receptor & time
keys   = ['X', 'Y', 'Z', 'T']
merged = conc_mod.merge(conc_ch, on=keys, suffixes=('_mod', '_ch'))

rtol, atol = 1e-8, 0                # ← relaxed but still very strict
if not np.allclose(merged['S_mod'], merged['S_ch'], rtol=rtol, atol=atol):
    diff = (merged['S_mod'] - merged['S_ch']).abs()
    print(f"⚠️  max |ΔS| = {diff.max():.3e} (rtol={rtol})")
    raise AssertionError("Patched and Chama results differ beyond tolerance.")
print("✅  Patched model and original Chama results match within tolerance.")




✅  Patched model and original Chama results match within tolerance.


In [19]:
import numpy as np
import pandas as pd

# conc_ch  : DataFrame from *original* Chama model  (ground-truth)
# conc_mod : DataFrame from *modified* model        (to be checked)

# ── 1. align both outputs on their common space-time keys ─────────────
keys   = ['X', 'Y', 'Z', 'T']                     # exact-match columns
merged = conc_ch.merge(conc_mod, on=keys, suffixes=('_ch', '_mod'))

# ── 2. per-receptor relative error (abs diff / true value) ────────────
#      If the true value is zero, set relative error to NaN so it
#      doesn’t blow up the statistics.
merged['rel_err'] = np.where(
    merged['S_ch'] != 0,
    np.abs(merged['S_mod'] - merged['S_ch']) / np.abs(merged['S_ch']),
    np.nan
)

# ── 3. quick summary ──────────────────────────────────────────────────
print("max   relative error :", merged['rel_err'].max())
print("mean  relative error :", merged['rel_err'].mean())
print("95th-percentile err  :", merged['rel_err'].quantile(0.95))


max   relative error : 2.310216009876513e-13
mean  relative error : 3.490781446558165e-14
95th-percentile err  : 1.4187549098765127e-13
